In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
print(train.shape)

train[['C-deck', 'C-num', 'C-side']] = train['Cabin'].str.split('/', expand=True)

train[['gggg','pp']] = train['PassengerId'].str.split('_', expand=True).astype(int)

train1 = train
train1['Count'] = 0
for i in set(train1['pp']):
    for index, row in train1.iterrows():
        if row['pp'] == i:
            train1.loc[train1['gggg'] == row['gggg'], 'Count'] = i

train1.head()

In [ ]:
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
print(test.shape)

test[['C-deck', 'C-num', 'C-side']] = test['Cabin'].str.split('/', expand=True)

test[['gggg','pp']] = test['PassengerId'].str.split('_', expand=True).astype(int)

test1 = test 
test1['Count'] = 0
for i in set(test1['pp']):
    for index, row in test1.iterrows():
        if row['pp'] == i:
            test1.loc[test1['gggg'] == row['gggg'], 'Count'] = i
                
test1.head(5)

In [ ]:
#Check and print if any group(PassengerId) has been split in train and test sets 
for i in set(test1['gggg']):
    for j in set(train1['gggg']):
        if i == j:
            print(test1['gggg'])

In [ ]:
submission = pd.read_csv("../input/spaceship-titanic/sample_submission.csv")
submission.head(2)

In [ ]:
print(train1.isna().sum(),'\n')
print(test1.isna().sum())

In [ ]:
from sklearn.impute import SimpleImputer

imputer_cols = ["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck" ,"RoomService"]
STRATEGY = 'mean'
imputer = SimpleImputer(strategy=STRATEGY )
imputer.fit(train1[imputer_cols])
train1[imputer_cols] = imputer.transform(train1[imputer_cols])
test1[imputer_cols] = imputer.transform(test1[imputer_cols])
train1["HomePlanet"].fillna('Z', inplace=True)
test1["HomePlanet"].fillna('Z', inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_cols = ["HomePlanet", "CryoSleep","Cabin", "Destination" ,"VIP", "C-deck", "C-num", "C-side"]
def label_encoder(train,test,columns):
    for col in columns:
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)
        train[col] = LabelEncoder().fit_transform(train[col])
        test[col] =  LabelEncoder().fit_transform(test[col])
    return train, test

train2 ,test2 = label_encoder(train1,test1 ,label_cols)

In [ ]:
train2.head()

In [ ]:
train3 = train2.drop(["PassengerId","Name" ,"Cabin", "gggg", "pp"] , axis = 1)
test3 = test2.drop(["PassengerId","Name" ,"Cabin", "gggg", "pp"] , axis = 1)
X = train3.drop('Transported' , axis =1 )
y = train3['Transported']

In [ ]:
X = X.astype('float32')
X.head()

In [ ]:
#Normalise the data
from sklearn.preprocessing import StandardScaler

scale = StandardScaler().fit(X)

X_train_scaled = scale.transform(X)
X_test_scaled = scale.transform(test3)

In [ ]:
from keras.utils import np_utils

print("Shape before one-hot encoding: ", y.shape)
y_cat = np_utils.to_categorical(y)
print(y.head())
y_cat[0:5]

In [ ]:
from keras.models import Sequential, load_model
from keras.layers.core import Dropout, Activation
from keras.layers import LeakyReLU, BatchNormalization, Dense

model = Sequential()

model.add(Dense(1024, input_shape=(14,)))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.05))
#model.add(Activation('relu'))                            
model.add(Dropout(0.5))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.05))
#model.add(Activation('relu'))
model.add(Dropout(0.5))

# model.add(Dense(128))
# model.add(BatchNormalization())
# model.add(LeakyReLU(alpha=0.05))
# #model.add(Activation('relu'))
# model.add(Dropout(0.5))

# model.add(Dense(64))
# model.add(BatchNormalization())
# model.add(LeakyReLU(alpha=0.05))
# #model.add(Activation('relu'))
# model.add(Dropout(0.5))

model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()

In [ ]:
import tensorflow as tf

# lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
#     initial_learning_rate=0.001,
#     decay_steps=100,
#     decay_rate=100)

model.compile(loss='categorical_crossentropy', 
              metrics=['accuracy'], 
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001))      #tf.keras.optimizers.RMSprop(learning_rate=lr_schedule))

In [ ]:
history = model.fit(X_train_scaled, y_cat,
          epochs=200,
          verbose=2,
          validation_split=0.0)

In [ ]:
model.compile(loss='categorical_crossentropy', 
              metrics=['accuracy'], 
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001))

In [ ]:
history = model.fit(X_train_scaled, y_cat,
          epochs=100,
          verbose=2,
          validation_split=0.0)

In [ ]:
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,3))
ax1.plot(history.history['accuracy'])
#ax1.plot(history.history['val_accuracy'])
ax1.set_title('Accuracy')
ax1.set(xlabel='epoch', ylabel='accuracy')
ax1.legend(['train', 'val'])

ax2.plot(history.history['loss'])
#ax2.plot(history.history['val_loss'])
ax2.set_title('Loss')
ax1.set(xlabel='epoch', ylabel='rms')
ax2.legend(['train', 'val'])

In [ ]:
y_pred = model.predict(X_test_scaled)
b = np.argmax(y_pred, axis=1)

b[0:10]

In [ ]:
i = 0
val = 0
for i, val in enumerate(b):
    if val == 0:
        submission['Transported'][i] = 'False'
    else:
        submission['Transported'][i] = 'True'

In [ ]:
submission.to_csv('submission.csv', index=False)
submission.head(10)